O GSDMM, por outro lado, visa especificamente detectar tópicos em documentos menores e assume apenas um tópico por documento.

In [12]:
# gerais
import pandas as pd
from tqdm import tqdm
import numpy as np

#plot
import matplotlib.pyplot as plt  

from gensim import corpora

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from unidecode import unidecode
from string import punctuation

from collections import defaultdict

from gsdmm import MovieGroupProcess

In [2]:
df = pd.read_csv('./../../../datasets/fakeTelegram_SemTravaZap_BR_2022.csv')

In [3]:
df.head()

,Unnamed: 0,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,...,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,caracteres_especial_count
0,0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN,24
1,1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN,29
2,2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN,77
3,3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,...,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,8
4,4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,...,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637,0


In [4]:
ids_para_remover = ['b73c0b674b28d87375d78dcae2f1a21f','62b3aa83e4df122e48cb1f97583e403f',
                    '893874da7344daaa00b8b5b3dc295b59','38737f3c7a07586c4efbfa39aa345705',
                    'da4cbbc1b7b1192ad83bcdae613ac2c3', 'bf66a467459264e0d33e4c8518a82827',
                    '46e009035e6150d656d0c194db88fb07','811541f6ce49347d1798e8dd2d3cbf83',
                    '1d2a0a63333d12ad188e9e6f7f2e0419','7c379613d4da713fe959d8c7ddc11ce2',
                    'debe17fd10d504076d2df8682f63bfc6','e9713ae04a02a810d6f33dd956f42794',
                    'd6c77928db26721ce46aca2d549780f0','8e53464d3b01eea3e39d07e51ecbb1b4',
                    'bacd40da04dd7d13f646993bdcf8e79d','2273d1167a6212812d95dc8fadbae78e',
                    'cd3bb1bdf75be7595e6373171a5c2225','add8c1ba533c5e5450d92c061a5ee7bf',
                    'e198f90df1995528531dd43db0c935ea', '06dc9ac55ed64caab2bd97e9ab717302',
                    'c0110feb539d212836605b66192722dd','f71912700ac5331415408ce229681359',
                    '2ee692357a9c948351c43a9540e859ae', '25b66278176dabe814dfc25a405a2470',
                    '08b21cb1e7de74ef5fe1085230075523']

In [5]:
# Filtrar o DataFrame para remover as linhas com esses IDs
df_filtrado = df[~df['id_member_anonymous'].isin(ids_para_remover)]
df_filtrado

,Unnamed: 0,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,...,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,caracteres_especial_count
0,0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN,24
1,1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN,29
2,2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN,77
3,3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,...,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,8
4,4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,...,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557510,557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-...,True,True,False,...,5,2022-11-16 14:49:39.146502,0.1027,NaN,575796,Url,telegram,NaN,25e43b6a58b848c43ad5b5f9e979822a,32
557511,557582,2022-11-11 12:09:08,NaN,5b10d7739171149be6d9961e3350c071,657949d03e4088f6b332e2686ccd3221.jpg,url,https://youtu.be/8g1Vz9_0xVk,True,True,False,...,5,2022-11-16 14:49:39.847434,0.0000,NaN,1286443,Url,telegram,NaN,657949d03e4088f6b332e2686ccd3221,6
557512,557583,2022-11-11 12:09:47,NaN,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,...,5,2022-11-16 14:49:39.922279,0.0000,NaN,13294,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,14
557513,557584,2022-11-11 12:09:46,NaN,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,...,5,2022-11-16 14:49:39.992932,0.0000,NaN,1286444,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,14


In [6]:
# Removendo linhas com valores NaN na coluna 'text_content_anonymous'
df_filtrado = df_filtrado.dropna(subset=['text_content_anonymous'])

# Removendo trava_zap
df_filtrado = df_filtrado[df_filtrado['trava_zap'] == False]

len(df_filtrado)

443705

In [7]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('portuguese'))

custom_stopwords = {
    't', 'be', 'nao', 'youtu', 'vai', 'av', 'ja', 'to', 'the', 'this', 'i', 'and', 
    'you', 'y', 'www', 'sao', 'pois', 'contra', 'user', 'ai', 'so', 'gente', 'voce', 'of', 
    'ola', 'gift', 'card', 'kwaivideo', 'r', 'bom', 'q', 'vc', 'vcs', 'pra','ta', 'phone', 'ok', 'la',
    'sera', 'ha', 'aqui', 'ate', 'dia', 'mc', 'im', 'tmj', 'pix', 'g', 'diz', 'ti', 'etc', 'tudo', 
    'todo', 'toda', 'youtube', 'g1', 'm', 'instagram', 'fb', 'in', 'link', 'was', 'blocked', 'kk'
}

stop_words.update(custom_stopwords)

def preprocess_text(text):

    # Função para extrair e substituir o domínio da URL
    def substituir_dominios(texto):
        # Função para extrair e substituir o domínio da URL
        def extrair_dominio(url):
            # Remove o protocolo (http://, https://, etc.) e o "www." se presente
            dominio = re.sub(r'^https?://(?:www\.)?|www\.', '', url)
            # Remove o caminho e parâmetros da URL
            dominio = re.split(r'[/?#]', dominio)[0]
            # Retorna a parte principal do domínio (antes do primeiro ponto)
            return dominio.split('.')[0]

        # Substitui URLs por seus domínios principais
        return re.sub(r'https?://(?:www\.)?\S+|www\.\S+', lambda match: extrair_dominio(match.group(0)), texto)

    # Substituir domínios
    text = substituir_dominios(text)

    # Converte para minúsculas
    text = text.lower()

    # Remove acentos
    text = unidecode(text)

    #Remover Pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove URLs e menções
    #text = re.sub(r'http\S+|www\S+|https\S+|@\w+', '', text)

    # Substitui emojis repetidos por apenas um
    text = re.sub(r'([\U00010000-\U0010FFFF])\1+', r'\1', text)
    text = re.sub(r'([\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F780-\U0001F7FF]|[\U0001F800-\U0001F8FF]|[\U0001F900-\U0001F9FF]|[\U0001FA00-\U0001FA6F]|[\U0001FA70-\U0001FAFF])\1+', r'\1', text)


    # Remove espaços em branco extras (início ou final) e múltiplos espaços no meio do texto
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove pontuações e caracteres especiais
    #text = re.sub(r'[^\w\s]', '', text)

    # Ajusta risadas "kkk" ou mais para "kk"
    text = re.sub(r'k{2,}|K{2,}', 'kk', text)

    # Ajusta risadas "haha" ou mais para "haha"
    text = re.sub(r'(ha){2,}', 'haha', text, flags=re.IGNORECASE)

    # Ajusta risadas "kaka" ou mais para "kaka"
    text = re.sub(r'(ka){2,}', 'kaka', text, flags=re.IGNORECASE)

    # Remove as stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])


    return text


[nltk_data] Downloading package punkt to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:

df_geral = df_filtrado.copy()

# Aplicar o pré-processamento à coluna de texto
df_geral['text_processed'] = df_geral['text_content_anonymous'].apply(preprocess_text)

In [9]:
docs = df_geral['text_processed'].apply(lambda x: x.split()).tolist()

In [10]:

vocab = defaultdict(int)
for doc in docs:
    for word in doc:
        vocab[word] += 1

vocab_size = len(vocab)
print(f"Tamanho do vocabulário: {vocab_size}")


Tamanho do vocabulário: 186256


In [11]:

# Parâmetros
K = 30   # Número máximo de tópicos (pode testar valores como 10, 15, 30...)
alpha = 0.1  # Distribuição de documentos por tópico
beta = 0.3   # Distribuição de palavras por tópico
n_iters = 30

# Instanciar e treinar
mgp = MovieGroupProcess(K=K, alpha=alpha, beta=beta, n_iters=n_iters)
y = mgp.fit(docs, vocab_size)


In stage 0: transferred 422669 clusters with 30 clusters populated
In stage 1: transferred 250831 clusters with 30 clusters populated
In stage 2: transferred 136770 clusters with 30 clusters populated
In stage 3: transferred 105949 clusters with 30 clusters populated
In stage 4: transferred 96226 clusters with 30 clusters populated
In stage 5: transferred 92847 clusters with 30 clusters populated
In stage 6: transferred 92158 clusters with 30 clusters populated
In stage 7: transferred 91106 clusters with 30 clusters populated
In stage 8: transferred 90927 clusters with 30 clusters populated
In stage 9: transferred 90326 clusters with 30 clusters populated
In stage 10: transferred 90380 clusters with 30 clusters populated
In stage 11: transferred 90263 clusters with 30 clusters populated
In stage 12: transferred 90726 clusters with 30 clusters populated
In stage 13: transferred 89919 clusters with 30 clusters populated
In stage 14: transferred 90299 clusters with 30 clusters populated
I

In [13]:
doc_count = np.array(mgp.cluster_doc_count)
print("Número de documentos por tópico:")
for i, count in enumerate(doc_count):
    print(f"Tópico {i}: {count} documentos")

Número de documentos por tópico:
Tópico 0: 131549 documentos
Tópico 1: 12847 documentos
Tópico 2: 47300 documentos
Tópico 3: 5007 documentos
Tópico 4: 1439 documentos
Tópico 5: 2411 documentos
Tópico 6: 3547 documentos
Tópico 7: 15581 documentos
Tópico 8: 4132 documentos
Tópico 9: 1118 documentos
Tópico 10: 16688 documentos
Tópico 11: 84739 documentos
Tópico 12: 11048 documentos
Tópico 13: 720 documentos
Tópico 14: 2810 documentos
Tópico 15: 1608 documentos
Tópico 16: 20882 documentos
Tópico 17: 1527 documentos
Tópico 18: 11492 documentos
Tópico 19: 20083 documentos
Tópico 20: 1453 documentos
Tópico 21: 1599 documentos
Tópico 22: 933 documentos
Tópico 23: 1595 documentos
Tópico 24: 1122 documentos
Tópico 25: 757 documentos
Tópico 26: 9167 documentos
Tópico 27: 686 documentos
Tópico 28: 1127 documentos
Tópico 29: 28738 documentos


In [14]:
def top_words(cluster_word_distribution, topic, top_n=10):
    word_counts = cluster_word_distribution[topic]
    return sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:top_n]

# Mostrar os top 10 tópicos com mais documentos
top_topics = doc_count.argsort()[-10:][::-1]

for topic in top_topics:
    print(f"\nTópico {topic}:")
    for word, count in top_words(mgp.cluster_word_distribution, topic, top_n=10):
        print(f"{word}: {count}")



Tópico 0:
bolsonaro: 18535
povo: 15055
vamos: 13848
presidente: 12818
brasil: 12488
estao: 11141
todos: 11020
agora: 8854
grupo: 7592
fazer: 7522

Tópico 11:
bolsonaro: 18210
lula: 14008
terrabrasilnoticias: 7162
brasil: 7152
video: 6653
tse: 5810
pt: 5499
twitter: 5477
presidente: 5258
sobre: 5131

Tópico 2:
telegram: 14339
welcome: 14329
professional: 14169
tool: 14169
managing: 14169
groups: 14169
video: 10054
foto: 7958
gazetabrasil: 1205
roughsex: 1133

Tópico 29:
13: 21960
lula: 21853
bolsonaro: 20691
brasil: 17528
intervencao: 16661
presidente: 13634
militar: 13080
todos: 11699
grupo: 11022
55: 10848

Tópico 16:
tse: 17461
brazil: 17430
superior: 17421
following: 17421
decision: 17421
community: 17421
electoral: 17421
court: 17421
is: 518
baie: 386

Tópico 19:
plataforma: 6608
parabens: 5542
investimento: 4597
obrigado: 3811
todos: 3632
empresa: 3482
agora: 3253
investir: 3023
lucro: 2784
feliz: 2529

Tópico 10:
rua: 5300
tmealexeconomia: 2763
fonte: 2193
canal: 1627
eua: 1460


## Calculando a coerência

In [15]:

# Criação do dicionário
dictionary = corpora.Dictionary(docs)

# Criação do corpus em formato BoW
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Extrair tópicos: lista de listas com palavras
topics = []
for topic_num in range(mgp.K):
    if mgp.cluster_doc_count[topic_num] == 0:
        continue  # Ignora tópicos vazios
    top_words_topic = top_words(mgp.cluster_word_distribution, topic_num, top_n=10)
    topic_words = [word for word, _ in top_words_topic]
    topics.append(topic_words)

from gensim.models import CoherenceModel

# c_v
coherence_cv = CoherenceModel(
    topics=topics,
    texts=docs,
    dictionary=dictionary,
    coherence='c_v'
).get_coherence()

# u_mass
coherence_umass = CoherenceModel(
    topics=topics,
    corpus=corpus,
    dictionary=dictionary,
    coherence='u_mass'
).get_coherence()

# c_uci
coherence_uci = CoherenceModel(
    topics=topics,
    texts=docs,
    dictionary=dictionary,
    coherence='c_uci'
).get_coherence()

# c_npmi
coherence_npmi = CoherenceModel(
    topics=topics,
    texts=docs,
    dictionary=dictionary,
    coherence='c_npmi'
).get_coherence()

# Exibir resultados
print(f"Coherence C_V: {coherence_cv:.4f}")
print(f"Coherence U_Mass: {coherence_umass:.4f}")
print(f"Coherence C_UCI: {coherence_uci:.4f}")
print(f"Coherence C_NPMI: {coherence_npmi:.4f}")


Coherence C_V: 0.5741
Coherence U_Mass: -4.3430
Coherence C_UCI: -0.9253
Coherence C_NPMI: 0.0992


## Diversidade dos tópicos

In [16]:
def topic_diversity_gsdmm(model, top_n=10):
    topic_words = []
    for k in range(model.K):
        if model.cluster_doc_count[k] == 0:
            continue  # Pula tópicos vazios
        words = [word for word, _ in top_words(model.cluster_word_distribution, k, top_n=top_n)]
        topic_words.extend(words)
    unique_words = set(topic_words)
    return len(unique_words) / (top_n * len(topic_words) // top_n)


In [17]:
import rbo

def compute_irbo_gsdmm(model, top_n=10):
    topics = []
    for k in range(model.K):
        if model.cluster_doc_count[k] == 0:
            continue
        topic_terms = [word for word, _ in top_words(model.cluster_word_distribution, k, top_n=top_n)]
        topics.append(topic_terms)

    # Calcular iRBO (1 - RBO médio entre pares de tópicos)
    n = len(topics)
    total_irbo = 0
    count = 0

    for i in range(n):
        for j in range(i + 1, n):
            rbo_score = rbo.RankingSimilarity(topics[i], topics[j]).rbo()
            total_irbo += (1 - rbo_score)
            count += 1

    return total_irbo / count if count > 0 else 0


In [18]:
diversidade = topic_diversity_gsdmm(mgp, top_n=10)
irbo = compute_irbo_gsdmm(mgp, top_n=10)

print(f"Topic Diversity: {diversidade:.4f}")
print(f"Inverted RBO (iRBO): {irbo:.4f}")


Topic Diversity: 0.7400
Inverted RBO (iRBO): 0.9622


In [21]:

# Lista para armazenar os tópicos
topics_data = []

# Você pode definir quantos tópicos quer salvar 
num_topicos_para_salvar = 30
top_topics = doc_count.argsort()[-num_topicos_para_salvar:][::-1]

# Para cada tópico, extrair palavras mais frequentes
for topic in top_topics:
    top_palavras = top_words(mgp.cluster_word_distribution, topic, top_n=10)
    palavras = ", ".join([word for word, _ in top_palavras])
    frequencias = ", ".join([str(count) for _, count in top_palavras])
    topics_data.append({
        "topico": topic,
        "palavras_mais_representativas": palavras,
        "frequencias": frequencias,
        "n_documentos": doc_count[topic]
    })

# Criar DataFrame
df_topicos = pd.DataFrame(topics_data)

# Salvar como CSV
df_topicos.to_csv("../resultados/gsdmm_telegram_sem_filtro.csv", index=False, encoding='utf-8')


In [22]:
df_topicos.head(10)

,topico,palavras_mais_representativas,frequencias,n_documentos
0,0,"bolsonaro, povo, vamos, presidente, brasil, es...","18535, 15055, 13848, 12818, 12488, 11141, 1102...",131549
1,11,"bolsonaro, lula, terrabrasilnoticias, brasil, ...","18210, 14008, 7162, 7152, 6653, 5810, 5499, 54...",84739
2,2,"telegram, welcome, professional, tool, managin...","14339, 14329, 14169, 14169, 14169, 14169, 1005...",47300
3,29,"13, lula, bolsonaro, brasil, intervencao, pres...","21960, 21853, 20691, 17528, 16661, 13634, 1308...",28738
4,16,"tse, brazil, superior, following, decision, co...","17461, 17430, 17421, 17421, 17421, 17421, 1742...",20882
5,19,"plataforma, parabens, investimento, obrigado, ...","6608, 5542, 4597, 3811, 3632, 3482, 3253, 3023...",20083
6,10,"rua, tmealexeconomia, fonte, canal, eua, sobre...","5300, 2763, 2193, 1627, 1460, 1373, 1232, 1225...",16688
7,7,"is, congratulations, are, have, your, company,...","5757, 3883, 3751, 3550, 3431, 3426, 3340, 2926...",15581
8,1,"bolsonaro, 22, presidente, vamos, governador, ...","6675, 4473, 4193, 1752, 1737, 1663, 1459, 1366...",12847
9,18,"ganhar, comecar, dinheiro, bot, pode, ativar, ...","10461, 9843, 8728, 7636, 7298, 7165, 6655, 646...",11492
